# Introduction

Word2Vec is a recently developed family of algorithms for generating distributed representations of words from a large corpus of text. These distributed representations are sometimes referred to as 'word embeddings' because the algorithms that generate them are designed to embed a vocabulary of words in a relatively low dimensional vector space (i.e. the dimensionality of the embeddings is usually on the order of hundreds, while the size of the vocabulary is usually on the order of tens or hundreds of thousands). Well known methods for creating word embeddings include LSA, HAL, BEAGLE, GloVe, and Word2Vec. Word2Vec's popularity as an embedding model is due to the fact that the representations it produces are able capture interesting relationships involving multiple words. 

Interestingly, Word2Vec uses two distinct model architectures: the CBOW (continuous-bag-of-words) architecture and the Skip-gram architecture. To explain, the CBOW architecture learns to predict a target word given a 'bag' or context of surrounding words. The Skip-gram architecture, on the other hand, does exactly the opposite - it learns to predict a bag of surrounding words given a target word. The figure below, taken from Mikolov et al. (2013), provides a nice visual depiction of each architecture, where $w$ indicates a word, and $t$ indicates the target word position.  

<br>
<center><img src="images/cbow.png" width=500px></center>  
<br>

The rest of this notebook describes each model in some mathematical detail, drawing on explanations provided by Rong (2015). The notebook also provides example applications of each algorithm to a corpus of approximately 2500 wikipedia articles. Optimizations concerning the use of hierarchical softmax and negative sampling are also discussed. 

# Part 1: Continuous Bag of Words (CBOW)

As illustrated in the figure above, the CBOW architecture is akin to a three layer neural network. The input layer encodes a binary vector indicating a bag of words (order is not important, so the $w(t-1)...$ notation in the figure merely indicates the window around the target word from which the bag is drawn), and the hidden layer maps the input layer into a low dimensional space without applying a non-linearity. Finally, the hidden layer is mapped to a softmax output layer that gives a probability distribution over the words in the model's vocabulary. 

In the simplest case, the input is a single word drawn from a corpus, and the target is the next word in the corpus. The input is encoded as a "one-hot" vector, and hence it extracts a single column from the input-to-hidden weight matrix ($W_1$). Thus, the value of the hidden layer is $h = W_1 x$, where $x$ is the input vector. Next, each unit in the output layer encodes a dot product between $h$ and a row in the hidden-to-output weight matrix ($W_2$). The softmax function is then used to convert these dot products into a proper probability distribution. It is important to highlight that each column of $W_1$ is vector representation of a particular word, as is each row of $W_2$. (So each word has *two* vector representations). The goal of learning is to make the $W_1$ vector representation of a word most similar to the $W_2$ vector representation of the word that is most likely to follow it in the corpus. 

Alternatively, one can think of the model as performing a variant of logistic regression using a factored weight matrix. The hidden layer does not include an element-wise non-linearity, so a weight matrix between the input and output layers is factored into two $N x V$ and $V x N$ components, where $V$ is the size of the vocabulary, and $N$ is the dimensionality of the hidden layer. These component matrices each contain a distributed representation of each word in the vocabulary - call them $v_w$ and $v_{w}^{'}$ for word $w$. So, the activation of the output layer after applying the softmax is as follows for input word $i$ and output word $o$:

$y_o = \frac{e^{v_i^{T} v_o^{'}}}{\sum_{j=1}^{V} e^{v_i^{T} v_j^{'}}}$

Learning is conducted by applying gradient descent to minimize a cost function defined in terms of the negative log-likelihood of the correct output word $o$. This cost function is similar to the one used previously for multiclass logistic regression (see the notebook on backpropogation):

$J(\theta) = -log(\frac{e^{v_i^{T} v_o^{'}}}{\sum_{j=1}^{V} e^{v_i^{T} v_j^{'}}}) = - v_i^{T} v_o^{'} + log\sum_{j=1}^{V} e^{v_i^{T} v_j^{'}}$  

where again $j$ is the index of the correct output word, $i$ is the index of the input word, and $V$ is the size of the vocabulary. Differentiating this cost function with respect to the total input to each output unit gives the familiar softmax derivative:

$ \frac{\partial J(\theta)}{\partial v_i^{T} v_j^{'}} = \frac{\partial J(\theta)}{\partial y_j} \frac{\partial y_j}{\partial v_i^{T} v_j^{'}} =  y_j - t_j $

where $t_j$ is the target output for unit $j$, and $y_j$ is the observed output. With this derivative, we can compute the gradient of the weights in the network using backprogation. At an intuitive level, the updates to $W_2$ are easy to understand. Each row $j$ in $W_2$ is updated with values of the hidden layer activities, but scaled by both the learning rate and the prediction error at $y_j$ (i.e. the softmax derivative). So, for a given output unit, if the model incorrectly guesses that the word corresponding to this unit ought to be predicted, the unit's incoming weights will be decremented to be more *dissimilar* to the hidden layer vector. In effect, this lowers the value of the dot product between the first layer representation of the input word, and the second layer representation of the output word under consideration. On the other hand, if the model incorrectly fails to predict the correct word, the unit corresponding to this word will have its incoming weights incremented to become more *similar* to the hidden layer vector. This raises the value of the dot product between the first layer representation of the input word and the second layer representation of the correct output word. As such, when same input is provided again to the model, it will be more likely to predict the correct output. The updates to $W_1$ follow a similar but less directly interpretable pattern. Finally, in cases where the input is multiple words instead of single word, the hidden layer simply encodes a sum of the column vectors in $W_1$ that correspond to each input word. The training objective and parameter update procedure remains the same.

With a large-sized vocabulary, CBOW is actually quite slow to train, mostly due to the computation of the softmax function on the output layer. We can see this by training a model on a few thousand documents from wikipedia. Training is performed by attempting to predict each word in each sentence of every document from a bag of up to six surrounding words.

First, it is necessary to build a vocabulary by getting word counts from a collection of documents. Words with low counts will be ignored to avoid idiosyncratic mispellings and extremely rare words; the remaining words will form the vocabulary. Counting words can be sped up using python's multiprocessing library as follows:

In [15]:
import collections
import multiprocessing
from utils import docstream, countwords

counts = collections.Counter()  
pool = multiprocessing.Pool()

# Stream n files from wikipedia dump and count words in parallel
for dlist in docstream(size=40):
    results = pool.map_async(countwords, dlist)
    for r in results.get():
        counts.update(r)
 
vocab = [x[0] for x in counts.iteritems() if x[1] > 20]
print 'Total Number of Words: ', sum(counts.values())
print 'Vocabulary Size: ', len(vocab)

Total Number of Words:  6289575
Vocabulary Size:  17878


Now it is possible define the CBOW model using a variation on the neural network class used in other notebooks. Some helper functions for grabbing contexts from sentences and converting words into one-hot vectors have been defined elsewhere to keep things manageably concise. We'll get access to these functions by inheriting from an EmbeddingModel class, and we'll define a base Word2Vec class to avoid duplicating code when defining the CBOW and Skipgram models:

In [16]:
import numpy as np
from embedding import EmbeddingModel

class Word2Vec(EmbeddingModel):
    """
    A shallow neural network that implements the learning algorithms
    for word embeddings described in Mikolov et al (2013). 
    
    Parameters:
    -----------
    voc : dict
        The vocabulary of words that embeddings are being learned for.
    dim : int 
        The dimensionality of the hidden layer of the network.
    eps : float, optional
        Scaling factor on random weight initialization. By default, the 
        weightsare chosen from a uniform distribution on the interval 
        [-0.1, 0.1].
    """
    def __init__(self, vocab, dim, eps=0.1):
        self.w1 = np.random.random((dim, len(vocab)))*eps*2-eps
        self.w2 = np.random.random((len(vocab), dim))*eps*2-eps
        self.word_indices = {j:i for i,j in enumerate(vocab)}
        self.win_size = 3
        self.vocab = vocab
            
    def get_activations(self):
        raise NotImplementedError('Models must compute activation values')
    
    def get_wordvecs(self):
        # Create word vectors from network weights
        self.word_vecs = self.w1.T + self.w2
        norms = np.linalg.norm(self.word_vecs, axis=1)
        self.word_vecs = np.divide(self.word_vecs, norms.reshape(len(norms), 1))

To define the CBOW model, we just need to implement methods for computing activations in the network, and for training the parameters through gradient descent: 

In [17]:
class CBOW(Word2Vec):
    """
    Implementation of the continuous-bag-of-words portion of 
    Word2Vec. This implementation is slow to train.
    """
    def get_activations(self, xs):
        self.yh = np.dot(self.w1, xs)
        self.yo = self.softmax(np.dot(self.w2, self.yh))

    def train(self, ndocs, rate=0.3): 
        for xs, ys in self.data(ndocs, model='cbow'):
            bsize = float(xs.shape[1])
            self.get_activations(xs)

            # Compute gradients               
            yo_grad = self.yo-ys
            yh_grad = np.dot(self.w2.T, yo_grad)

            w1_grad = np.dot(yh_grad, xs.T) / bsize
            w2_grad = np.dot(yo_grad, self.yh.T) / bsize

            # Update weights
            self.w1 += -rate * w1_grad
            self.w2 += -rate * w2_grad
            
        self.get_wordvecs()

To test the model, 2500 wikipedia documents are used for training. Keeping track of how long it takes to train the model is useful for determining the average rate at which it processes the wikipedia articles. 

In [18]:
import time
start_time = time.time()
ndocs = 2500

embedder = CBOW(vocab, dim=100) 
embedder.train(ndocs=ndocs)

print 'Average training time per document: ', (time.time()-start_time) / float(ndocs)

Average training time per document:  7.41550971403


Next we define a simple function for retrieving the nearest neighbors to a collection of query words. 

In [19]:
def query(model, terms):
    for term in terms:
        print 'Nearest neighbors to "%s":' %term
        model.get_synonyms(term)
        print ''

terms = ['roman','medieval','car','french','english']
query(embedder, terms)

Nearest neighbors to "roman":

roman 1.0
catholics 0.696445787939
carolingian 0.657715390353
anglicanism 0.644777226209
reformed 0.643190816476

Nearest neighbors to "medieval":

medieval 1.0
renaissance 0.712891192886
carolingian 0.695768596296
anglosaxon 0.679891606384
thcentury 0.659358089648

Nearest neighbors to "car":

car 1.0
cars 0.686071171259
vehicle 0.584242059647
motor 0.576982920912
driver 0.575006794875

Nearest neighbors to "french":

french 1.0
german 0.740261812999
spanish 0.698222116719
colonists 0.687998349914
english 0.67328398971

Nearest neighbors to "english":

english 1.0
breton 0.710170901501
vernacular 0.698236360063
welsh 0.68714843066
french 0.67328398971



# Part 2: Skip-gram

The Skip-gram model is essentially the CBOW model run in reverse. Rather than learn to predict a word from a context, the model learns to predict a context from a word. This means that the input layer of the model encodes a single one-hot vector, and the output layer encodes a probability distribution over the model's vocabulary, just as before. The training objective changes slightly, though, since the goal is to have each input word assign high probabilities to multiple items in the vocabulary. To define the training objective, it is helpful to first imagine that there is output layer for each word in the context that is being predicted. Since a single set of weights is used to compute the probability distributions on these output layers, they all have activation values defined by the previously used softmax equation, where $c$ indexes the output layer for each context word:  

$y_{c, o} = \frac{e^{v_i^{T} v_o^{'}}}{\sum_{j=1}^{V} e^{v_i^{T} v_j^{'}}}$

The cost function is again similar to before. The main change is that the cost is minimized when the product of the probabilities corresponding to each word in the context is maximized:

$J(\theta) = -log \prod \limits_{c=1}^{C} \frac{e^{v_i^{T} v_{o_c}^{'}}}{\sum_{j=1}^{V} e^{v_i^{T} v_j^{'}}} = \sum \limits_{c=1}^{C} - v_i^{T} v_{o_c}^{'} + C \cdot log\sum_{j=1}^{V} e^{v_i^{T} v_j^{'}}$  

Calculating the derivative of this cost function with respect to the activities of the output layers involves computing the prediction error on each layer with respect to the appropriate context word. In other words, we compute the same derivative as before, but $C$ times for each of the $C$ context words: 

$ \frac{\partial J(\theta)}{\partial v_i^{T} v_j^{'}} = \frac{\partial J(\theta)}{\partial y_{j_c}} \frac{\partial y_{j_c}}{\partial v_i^{T} v_j^{'}} =  y_{j_c} - t_{j_c}$

Note that $y_{j_c}$ is the same for all $c$, but $t_{j_c}$ changes depending on which index $j$ corresponds to the target context word in context $c$. Because of this fact, it is possible to express the gradient of the cost with respect to the output activities as a single vector of length $V$ whose value is $C \cdot y - \sum \limits_{c=1}^{C} t_c$, where $y$ is predicted set of probabilities over the over the model's vocabulary given the input word, and $t_c$ is the target set of probabilities for context word $c$ (i.e. a onehot vector). In short, the gradient is a multiple of the output probabilities minus a binary vector that encodes the target words in the context that is being predicted. This gradient is defined for a single output layer (which is in fact the case, since the same weights are used to predict each of the $C$ context words), meaning that it can be used to compute gradients for hidden-to-output weights in the model, along with the activites of the hidden units. As such, it is now possible to train all of the parameters in the model using gradient descent as before.  

Here is an illustration that trains a skipgram model on the same selection of wikipedia articles used to train the CBOW model. 

In [20]:
class Skipgram(Word2Vec):
    
    def get_activations(self, xs, ys):
        self.yh = np.dot(self.w1, xs)  
        self.yo = self.softmax(np.dot(self.w2, self.yh))
        self.yo = self.yo * np.sum(ys, axis=0)
        
    def train(self, ndocs, rate=0.3): 
        for xs, ys in self.data(ndocs, model='skipgram'): 
            bsize = float(xs.shape[1])
            
            # Compute activations
            self.get_activations(xs, ys)
            
            # Compute gradients               
            yo_grad = self.yo-ys
            yh_grad = np.dot(self.w2.T, yo_grad)

            w1_grad = yh_grad / bsize # since input is onehot
            w2_grad = np.dot(yo_grad, self.yh.T) / bsize

            # Update weights
            w1_inds = np.argwhere(xs == 1)[:,0]
            self.w1[:,w1_inds] += -rate * w1_grad
            self.w2 += -rate * w2_grad
            
        self.get_wordvecs()

In [21]:
start_time = time.time()

embedder = Skipgram(vocab, dim=100) 
embedder.train(ndocs=ndocs)

print 'Average training time per document: ', (time.time()-start_time) / float(ndocs)

Average training time per document:  4.88276838598


For comparison, here are the nearest neighbors for the same words as before using the embeddings learned with the skipgram model. 

In [23]:
query(embedder, terms)

Nearest neighbors to "roman":

roman 1.0
catholic 0.743833867403
empire 0.67409600379
orthodox 0.667566752048
church 0.64145647129

Nearest neighbors to "medieval":

medieval 1.0
renaissance 0.520106445433
history 0.517335125021
islamic 0.515302105765
canon 0.504319746887

Nearest neighbors to "car":

car 1.0
racing 0.674024374125
cars 0.643920525892
passenger 0.563319049684
driver 0.542714205202

Nearest neighbors to "french":

french 1.0
german 0.675494891721
british 0.644410722725
army 0.572199986128
troops 0.554868078522

Nearest neighbors to "english":

english 1.0
scottish 0.575486324727
dutch 0.56137342481
italian 0.559791721142
native 0.552642212734



# Part 3: Optimizations via Negative Sampling

*NOTE:* The current implementation of this optimization does not provide a speedup (it is actually slower). I'm not entirely sure why, but I think it's because the model updates its weights independently for each target word instead of computing updates over a minibatch. The results are also pretty bad, perhaps due to a need for a larger amount of training data with negative sampling. In any eveny, updates are coming soon.

The time logging shown above makes it pretty clear that training these models is horrendously slow and unlikely to scale well to very large corpora. The main bottleneck is due to the size of the softmax layer, which is equal to the size of the vocabulary being modelled. In the case of the input-to-hidden weights, it is possible to get away with updating only a fraction of the weights, because the gradient is only non-zero for the columns of the weight matrix that correspond to the input words in a given training batch. But in the case of the hidden-to-output weights the gradient is defined for every row in the weight matrix, because the prediction error is defined for every output unit (i.e. for every word in the vocabulary). If the vocabulary is on the order of tens or hundreds of thousands of terms, then each training example requires computing the gradient for an $N x V$ matrix. This is expensive, and constitutes a well-known problem for the training of neural language models more generally. 

A relatively simple solution proposed by Mikolov et al. (2013) involves computing the prediction error on only a random subset of the output units that always includes the target output unit. The cost of the model's prediction on a particular training example is then defined using the standard cost function used in logistic regression (this is slightly different from the previous cost function, because the assignment of higher probabilities to non-target words is also penalized strongly, whereas the previous cost only penalized the assignment of low probabilities to target words). One way to think about what is going on here is that the model is treating the ouput layer as a much smaller, randomly selected set of units on each training iteration. Moreover, the values of these units do not define a well-formed probability distribution, so there is no need to go through the trouble of computing the softmax function. Rather, outputs are simply scalar values, and the model aims to boost the values that corresponds to the correct output words and shrink the other values. The standard logistic regresssion cost function that is described in a previous notebook suffices for this purpose:

$J(\theta) = -\sum_{j=1}^{K} [t_j \cdot log(\sigma(h^{T} v_j^{'})) + (1-t_j) \cdot log(1-\sigma(h^{T} v_j^{'}))]$

Note this defines the cost on a single training example; to calculate the total cost, one would sum this value over the entire set of training data. Also note that the correct output unit must be included in the $K$ units over which the cost is defined. The derivative of this cost function with respect to the input to an output unit $j$ is as follows:

$\frac{\partial J(\theta)}{\partial h^{T}v_j^{'}} = \sigma(h^{T}v_j^{'}) - t_j$

And with respect to the output word vector $v_j^{'}$ (i.e. a selection of weights in the network), the derivative is:

$\frac{\partial J(\theta)}{\partial h^{T}v_j^{'}} \frac{\partial h^{T}v_j^{'}}{\partial v_j^{'}} = h^{T} (\sigma(h^{T}v_j^{'}) - t_j)$

We can also compute the derivative of the cost function with respect to the activities in the hidden layer, and therefore compute the necessary derivatives with respect to the input-to-hidden weights. This is all we need to implement gradient descent as before. 

In [25]:
class CBOWns(Word2Vec):
    """
    A CBOW model with negative sampling for efficient parameter
    updates. Currently this is a work in progress.
    """
    def get_activations(self, x, inds):
        self.yh = np.dot(self.w1, x)
        self.yo = self.sigmoid(np.dot(self.w2[inds,:], self.yh))
        
    def train(self, ndocs, rate=0.15, k=25): 
        for x, y in self.ns_data(ndocs, model='cbow'):
            targ = np.where(y==1)[0][0]
            inds = np.append(np.random.choice(len(vocab), k), targ)
            inds = np.unique(inds)
            self.get_activations(x, inds)

            # Compute gradients               
            yo_grad = (self.yo-y[inds])            
            yh_grad = np.dot(self.w2[inds,:].T, yo_grad)

            w1_grad = np.outer(yh_grad, x) 
            w2_grad = np.outer(yo_grad, self.yh) 
            
            # Update weights
            self.w1 += -rate * w1_grad
            self.w2[inds,:] += -rate * w2_grad
            
        self.get_wordvecs()

In [26]:
import time
start_time = time.time()

embedder = CBOWns(vocab, dim=100) 
embedder.train(ndocs=ndocs)

print 'Average time per document: ', (time.time()-start_time) / ndocs

Average time per document:  14.5631294196


And again for comparison, here are the nearest neighbors for the same words as before using the embeddings learned with the CBOW model using negative sampling. 

In [27]:
query(embedder, terms)

Nearest neighbors to "roman":

roman 1.0
catholic 0.453925567649
gesta 0.402358997513
edible 0.390034642158
kerry 0.384136566826

Nearest neighbors to "medieval":

medieval 1.0
bp 0.462081358492
oslo 0.415294276363
jews 0.408853174532
everyday 0.399435803103

Nearest neighbors to "car":

car 1.0
marginleftem 0.398814364161
national 0.38481038144
system 0.352736420605
operators 0.350481695103

Nearest neighbors to "french":

french 1.0
logistical 0.657657834304
regularly 0.455847321547
patrol 0.453180191973
protests 0.443635697868

Nearest neighbors to "english":

english 1.0
mixing 0.450523046377
british 0.436482129213
ashore 0.416235646498
gratitude 0.393002273951



# Part 4: Optimizations via Hierarchical Softmax

Content in progress...